# SQuAD and MNLI on IPUs using DeBERTa - Inference

This notebook provides an implementation of two natural language understanding (NLU) tasks using small, efficient models: [Microsoft DeBERTa-base](https://arxiv.org/abs/2006.03654) for sequence classification and question answering. The notebook demonstrates how these models can achieve good performance on standard benchmarks while being relatively lightweight and easy to use. 

The two NLU tasks covered in this notebook are:
- Multi-Genre Natural Language Inference (MNLI) - a sentence-pair classification task
- Stanford Question Answering Dataset (SQuAD) - a question answering task

Hardware requirements: The models show each DeBERTa Base model running on two IPUs. If correctly configured, these models could both be served simultaneously on an IPU POD4.

[![Join our Slack Community](https://img.shields.io/badge/Slack-Join%20Graphcore's%20Community-blue?style=flat-square&logo=slack)](https://www.graphcore.ai/join-community)

##### Optimum Graphcore
The notebook also demonstrates [Optimum Graphcore](https://github.com/huggingface/optimum-graphcore). Optimum Graphcore is the interface between the Hugging Face Transformers library and [Graphcore IPUs](https://www.graphcore.ai/products/ipu). This notebook demonstrates a more explicit way of using Huggingface models with the IPU. This method is particularly useful when the task in question is not supported by the Huggingface pipelines API.

The easiest way to run a Huggingface inference model would be to instantiate the pipeline as follows:

```
oracle = pipeline(model="Palak/microsoft_deberta-base_squad")
oracle(question="Where do I live?", context="My name is Wolfgang and I live in Berlin")
```

However in some cases such as MNLI, there is no off-the-shelf pipeline ready to use. In this case, you could simply:
- Instantiate the model with the correct execution mode
- Use the optimum-specific call `to_pipelined` to return the model with changes and annotations for running on the IPU
- Set the model to run in `eval` mode and use the `parallelize` method on the new model to parallelize it across IPUs
- Prepare it for inference using `poptorch.inferenceModel()`

```
model = DebertaForQuestionAnswering.from_pretrained("Palak/microsoft_deberta-base_squad")

ipu_config = IPUConfig(ipus_per_replica=2, matmul_proportion=0.2, executable_cache_dir="./exe_cache")
pipelined_model = to_pipelined(model, ipu_config).eval().parallelize()
pipelined_model = poptorch.inferenceModel(pipelined_model, options=ipu_config.to_options(for_inference=True))
```

This method is demoed in this notebook, as Huggingface do not natively support the MNLI inference task.

In order to improve usability and support for future users, Graphcore would like to collect information about the
applications and code being run in this notebook. The following information will be anonymised before being sent to Graphcore:

- User progression through the notebook
- Notebook details: number of cells, code being run and the output of the cells
- Environment details

You can disable logging at any time by running `%unload_ext graphcore_cloud_tools.notebook_logging.gc_logger` from any cell.

## Setup
Install the optimum library

In [ ]:
%pip install "optimum-graphcore==0.7"
%pip install graphcore-cloud-tools[logger]@git+https://github.com/graphcore/graphcore-cloud-tools
%load_ext graphcore_cloud_tools.notebook_logging.gc_logger

We read some configuration from the environment to support environments like Paperspace Gradient.

In [ ]:
import os

executable_cache_dir = os.getenv("POPLAR_EXECUTABLE_CACHE_DIR", "./exe_cache")

Imports

In [ ]:
import os
import torch
from datasets import load_dataset, Dataset

import poptorch
from optimum.graphcore import IPUConfig
from optimum.graphcore.modeling_utils import to_pipelined

from transformers import BartForConditionalGeneration, BartTokenizerFast
from transformers import DebertaForSequenceClassification, DebertaTokenizerFast
from transformers import DebertaForQuestionAnswering, AutoTokenizer

## Multi-Genre Natural Language Inference (MNLI)

MNLI is a sentence-pair classification task, where the goal is to predict whether a given hypothesis is true (entailment) or false (contradiction) given a premise. The task has been proposed as a benchmark for evaluating natural language understanding models. 

In this notebook, we use the Microsoft DeBERTa-base model to classify pairs of sentences on the MNLI task. We first load the model and the tokenizer, then prepare an example input. Finally, we execute the model on an IPU device using PopTorch and obtain the predicted probabilities for the entailment classes.


First, load the model and tokeniser from the Huggingface Model Hub

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base-mnli")
model = DebertaForSequenceClassification.from_pretrained("microsoft/deberta-base-mnli")
model.half()

Create some example inputs, and encoder those using the tokeniser

In [ ]:
premise = "A man inspects the uniform of a figure in some East Asian country."
hypothesis = "The man is in an East Asian country."

inputs = tokenizer.encode(
    premise, hypothesis, return_tensors="pt", truncation_strategy="only_first"
)

Configure the instantiated model to run on IPUs

In [ ]:
ipu_config = IPUConfig(ipus_per_replica=2, matmul_proportion=0.6, executable_cache_dir=executable_cache_dir)
pipelined_model = to_pipelined(model, ipu_config).eval().parallelize()
pipelined_model = poptorch.inferenceModel(pipelined_model, options=ipu_config.to_options(for_inference=True))

Run the MNLI model and print the probability of entailment. We calculate this by throwing away neutral (index 1) and running softmax over the remaining logits.

In [ ]:
logits = pipelined_model(inputs)[0]
entail_contradiction_logits = logits[:, [0, 2]]
prob_label_is_true = entail_contradiction_logits.softmax(dim=1)[:, 1]
print(prob_label_is_true)

## Stanford Question Answering Dataset (SQuAD)

SQuAD is a question answering dataset consisting of questions posed on a set of Wikipedia articles. The goal is to answer the questions by highlighting the span of text in the corresponding article. 

In this notebook, we use the Microsoft DeBERTa-base model to answer a question given a context paragraph. We first load the model and the tokenizer, then prepare an example input. Finally, we execute the model on an IPU device using PopTorch and obtain the predicted answer span.


First, load the model and tokeniser from the Huggingface Model Hub

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Palak/microsoft_deberta-base_squad")
model = DebertaForQuestionAnswering.from_pretrained(
    "Palak/microsoft_deberta-base_squad"
)
model.half()

Create some example inputs, and tokenise them:

In [ ]:
question = "What's my name?"
context = "My name is Clara Smith and I live in Berkeley."

inputs = tokenizer.encode(
    question, context, return_tensors="pt", truncation_strategy="only_first"
)

Configure the instantiated model to run on IPUs

In [ ]:
ipu_config = IPUConfig(ipus_per_replica=2, matmul_proportion=0.2, executable_cache_dir=executable_cache_dir)
pipelined_model = to_pipelined(model, ipu_config).eval().parallelize()
pipelined_model = poptorch.inferenceModel(pipelined_model, options=ipu_config.to_options(for_inference=True))

Run the SQuAD model and print the identified span

In [ ]:
outputs = pipelined_model(inputs)
pred_tokens = inputs[0, outputs.start_logits.argmax() : outputs.end_logits.argmax() + 1]
print(tokenizer.decode(pred_tokens))

## Conclusion

DeBERTa is a powerful natural language understanding model that makes efficient use of compute resources and parameters. It is demonstrated in this notebook on textual entailment and question answering tasks - which are valuable building blocks for larger NLP systems.